In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-credit-default-risk/bureau.csv
/kaggle/input/home-credit-default-risk/HomeCredit_columns_description.csv
/kaggle/input/home-credit-default-risk/installments_payments.csv
/kaggle/input/home-credit-default-risk/bureau_balance.csv
/kaggle/input/home-credit-default-risk/application_train.csv
/kaggle/input/home-credit-default-risk/sample_submission.csv
/kaggle/input/home-credit-default-risk/POS_CASH_balance.csv
/kaggle/input/home-credit-default-risk/previous_application.csv
/kaggle/input/home-credit-default-risk/credit_card_balance.csv
/kaggle/input/home-credit-default-risk/application_test.csv


In [2]:
df_train = pd.read_csv('../input/home-credit-default-risk/application_train.csv')
df_test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')

Assign which ever datasets you want to train and test. This is because as part of feature engineering, you will often build new and different feature datasets and would like to test each one out to evaluate whether it improves model performance.

As the imputer is being fitted on the training data and used to transform both the training and test datasets, the training data needs to have the same number of features as the test dataset. This means that the TARGET column must be removed from the training dataset, and stored in train_labels for use later.

In [3]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_train['NAME_CONTRACT_TYPE'] = le.fit_transform(df_train['NAME_CONTRACT_TYPE'])
df_train['FLAG_OWN_CAR'] = le.fit_transform(df_train['FLAG_OWN_CAR'])
df_train['FLAG_OWN_REALTY'] = le.fit_transform(df_train['FLAG_OWN_REALTY'])

df_test['NAME_CONTRACT_TYPE'] = le.fit_transform(df_test['NAME_CONTRACT_TYPE'])
df_test['CODE_GENDER'] = le.fit_transform(df_test['CODE_GENDER'])
df_test['FLAG_OWN_CAR'] = le.fit_transform(df_test['FLAG_OWN_CAR'])
df_test['FLAG_OWN_REALTY'] = le.fit_transform(df_test['FLAG_OWN_REALTY'])

In [4]:
df_train = pd.get_dummies(df_train)
df_test = pd.get_dummies(df_test)

In [5]:
print(df_train.shape)
print(df_test.shape)

(307511, 243)
(48744, 238)


In [6]:
train_col = set(df_train.columns)
test_col = set(df_test.columns)

exc_col = train_col-test_col
print(exc_col)

{'CODE_GENDER_M', 'TARGET', 'NAME_INCOME_TYPE_Maternity leave', 'CODE_GENDER_F', 'NAME_FAMILY_STATUS_Unknown', 'CODE_GENDER_XNA'}


In [7]:
train_target_labels = df_train['TARGET']
df_train_align, df_test_align = df_train.align(df_test,join='inner',axis=1)
df_train_align['TARGET'] = train_target_labels

In [8]:
print('Training dataset shape: '.format(0),df_train_align.shape)
print('Testing dataset shape: '.format(0),df_test_align.shape)

Training dataset shape:  (307511, 238)
Testing dataset shape:  (48744, 237)


In [9]:
train = df_train_align.copy()
test = df_test_align.copy()

labels = train.pop('TARGET').values # store training labels
feat_names = list(train.columns) #

In [10]:
train_ids = train.index.values
test_ids = test.index.values

****FEATURE SET PREPROCESSING****

In [11]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan,strategy='median')

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range= (0,1))

We fit the imputer and scaler on the training data, and perform the imputer and scaling transformations on both the training and test datasets.

Scikit-learn models only accepts arrays. So the imputer and scalers can accept DataFrames as inputs and they output the train and test variables as arrays for use into Scikit-Learn's machine learning models.

In [12]:
imputer.fit(train)
train = imputer.transform(train)
test = imputer.transform(test)

scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

In [13]:
train.shape

(307511, 237)

**MODEL IMPLEMENTATION(GRADIETN BOOSTING MACHINE**
With the Gradient Boosting machine, we are going to perform an additional step of using K-fold cross validation (i.e., Kfold). In the other models (i.e., Logit, Random Forest) we only fitted our model on the training dataset and then evaluated the model's performance based on the test dataset.

Using Kfold, we are going to split up our test data set into multiple folds (i.e., KK). We will be fitting our model on K-1K−1 folds and testing it on the K^{th}K 
th
  fold (i.e., out-of-fold) and repeating the fitting process until all KK folds are explored. This method allows us to train our model more accurately without overfitting.

Thus, we copy our train (test_feat) dataframe to feat (test_feat) as these are more accurate descriptors since they are now feature and test feature datasets. The training dataset is split into further training and validation datasets based on the kfold.


In [14]:
feat = train.copy()
test_feat = test.copy()

In [15]:
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
import lightgbm as lgb
import gc

In [16]:
feat_imp_vals = np.zeros(len(feat_names)) # feature importance
test_pred = np.zeros(test_feat.shape[0]) # test predictions
oof = np.zeros(feat.shape[0]) # out of fold predictions
valid_scores = [] # validation scores
train_scores = [] # training scores
n_folds = 5

In [17]:
k_fold = KFold(n_splits = n_folds, shuffle=True,random_state=100)

With the KFold we are taking our training dataset, and splitting it into further training and validation datasets. The k_fold

n_estimators: Number of boosted trees to fit (i.e., 1000 trees).
reg_alpha: L1 regularization on weights (i.e., LASSO).
reg_lambda: L2 regularization on weights (i.e, Ridge.
subsample: Subsample ratio of training instance. Setting it to 0.5 lgb will randomly sample half of the training data prior to growing trees. and this will prevent overfitting. Subsampling will occur once in every boosting iteration. This prevents overfitting of the tree.
learning_rate: Boosting learning rate.
n_job : Number of processes to use for calculation. -1 means all processes will be used.
random_state : Allows model fit to be replicated.
class_weight :

In [18]:
for train_indices, valid_indices in k_fold.split(feat):
    
    train_feat, train_labels = feat[train_indices], labels[train_indices] # training data for the fold
    valid_feat, valid_labels = feat[valid_indices], labels[valid_indices] # validation data for the fold
    
    model = lgb.LGBMClassifier(n_estimators=1000, objective='binary',
                              class_weight='balanced',learning_rate=0.05,
                              reg_alpha = 0.1, reg_lambda=0.1,
                              subsample = 0.8, n_jobs = -1, random_state=50)


    model.fit(train_feat, train_labels, eval_metric = 'auc',
              eval_set = [(valid_feat, valid_labels), (train_feat, train_labels)],
              eval_names = ['valid','train'], early_stopping_rounds = 100, verbose = 200)
             # categorical_feature = cat_indices,
    
    # record best iteration of each fold
    best_iter = model.best_iteration_ 
    # record the most important features of each fold
    feat_imp_vals += model.feature_importances_/k_fold.n_splits 
    # record the out-of-fold predictions
    oof[valid_indices] = model.predict_proba(valid_feat, num_iteration = best_iter)[:,1]/k_fold.n_splits
    
    # record the predictions on the test_feat dataset
    test_pred += model.predict_proba(test_feat, num_iteration =  best_iter)[:,1]/k_fold.n_splits 
    
    valid_score = model.best_score_['valid']['auc']
    train_score = model.best_score_['train']['auc']
    
    valid_scores.append(valid_score)
    train_scores.append(train_score)
    
    gc.enable()
    del model, train_feat, valid_feat
    gc.collect()
    


Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.794899	train's binary_logloss: 0.552147	valid's auc: 0.757004	valid's binary_logloss: 0.567464
Early stopping, best iteration is:
[222]	train's auc: 0.798655	train's binary_logloss: 0.548504	valid's auc: 0.75711	valid's binary_logloss: 0.565247
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.795077	train's binary_logloss: 0.551788	valid's auc: 0.756384	valid's binary_logloss: 0.564369
Early stopping, best iteration is:
[258]	train's auc: 0.804371	train's binary_logloss: 0.542757	valid's auc: 0.756918	valid's binary_logloss: 0.558684
Training until validation scores don't improve for 100 rounds
[200]	train's auc: 0.794963	train's binary_logloss: 0.551713	valid's auc: 0.754765	valid's binary_logloss: 0.567765
Early stopping, best iteration is:
[245]	train's auc: 0.802327	train's binary_logloss: 0.544465	valid's auc: 0.754948	valid's binary_logloss: 0.563343
Training until

**Creating summary of validation and training AUC scores**

In [19]:
valid_auc = roc_auc_score(labels,oof) # calculate the auc based on the test dataset labels and the out-of-fold predictions
valid_scores.append(valid_auc) # calculate the overall validation auc score
train_scores.append(np.mean(train_scores)) # calculate the overall average training auc score

fold_names = list(range(n_folds))
fold_names.append('overall')

metrics = pd.DataFrame({'fold': fold_names,
                      'train': train_scores,
                      'valid': valid_scores})


In [23]:
import matplotlib.pyplot as plt
feat_imp = pd.DataFrame({'Feature': feat_names,'Importance':feat_imp_vals}) # creating feature importance dataframe
print(feat_imp.sort_values(by=['Importance'],ascending=False).head(6))


         Feature  Importance
31  EXT_SOURCE_1       549.2
33  EXT_SOURCE_3       499.4
32  EXT_SOURCE_2       438.4
10    DAYS_BIRTH       423.6
6     AMT_CREDIT       409.0
7    AMT_ANNUITY       367.2


In [24]:
submission = pd.DataFrame({'SK_ID_CURR': test_ids, 'TARGET': test_pred}) # creating Kaggle submission dataframe

In [25]:
submit = submission
print(submit.head())
print(submit.shape)

   SK_ID_CURR    TARGET
0           0  0.249140
1           1  0.470817
2           2  0.168403
3           3  0.232149
4           4  0.640061
(48744, 2)


In [26]:
submit.to_csv('lightgbm-home-loan-credit-risk.csv',index=False)

We review our Light GBM from Kaggle and find that there is a slight improvement to 0.74 compared to 0.662 (logit) or 0.688 (random-forest). We can see that substantial improvements are obtained using LightGBM with the same dataset as logit or random-forest leading us to understand why Gradient Boosted Machines are the machine learning model of choice for many data scientists.